# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Bin Ware

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

# Question 1

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [ ]:
import requests
import json

# Make the API request and get the JSON data
response = requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul')
result = response.json()

# Write the JSON data to a file
with open('lyrics.json', 'w') as file:
    json.dump(result, file)

# Question 2

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [ ]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load the lyrics from the JSON file
with open('your_lyrics_file.json', 'r') as file:
    data = json.load(file)

# Extract the lyrics
lyrics = data.get('lyrics', '')
print("Lyrics:\n", lyrics)

# Load spaCy model and add spaCyTextBlob to the pipeline
nlp = spacy.load('en_core_web_sm')
if 'spacytextblob' not in nlp.pipe_names:
    nlp.add_pipe('spacytextblob')

# Perform sentiment analysis on the lyrics
doc = nlp(lyrics)
polarity_score = doc._.polarity
print("\nPolarity Score:", polarity_score)

# Interpret the polarity score
if polarity_score > 0:
    print("The lyrics have a positive connotation.")
elif polarity_score < 0:
    print("The lyrics have a negative connotation.")
else:
    print("The lyrics are neutral.")


# Question 3

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [ ]:
import requests
import json

# Writing 8 songs to different files
songs = [
    {"artist": "Bruno Mars", "title": "Just the Way You Are"},
    {"artist": "The Weeknd", "title": "Blinding Lights"},
    {"artist": "Michael Jackson", "title": "Man in the Mirror"},
    {"artist": "Kanye West", "title": "Stronger"}
]

base_url = "https://api.lyrics.ovh/v1"

for song in songs:
    artist = song["artist"]
    title = song["title"]
    try:
        response = requests.get(f"{base_url}/{artist}/{title}")
        response.raise_for_status()  
        result = json.loads(response.text)

        # Create a unique file name for each song
        file_path = f"{title.replace(' ', '_')}_lyrics.json"

        with open(file_path, 'w') as file:
            json.dump(result, file, indent=4)

        print(f"Lyrics for '{title}' by {artist} have been written to {file_path}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch lyrics for '{title}' by {artist}: {e}")


# Question 4

4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [ ]:
def get_lyrics_polarity(file_name):
    with open(file_name, 'r') as file:
        data = json.load(file)
    
    lyrics = data.get('lyrics')
    if not lyrics:
        print(f"No lyrics found in {file_name}.")
        return None
    
    nlp = spacy.load("en_core_web_sm")
    if "spacytextblob" not in nlp.pipe_names:
        nlp.add_pipe("spacytextblob")
    
    doc = nlp(lyrics)
    return doc._.polarity

files = [
    "Just_the_Way_You_Are_lyrics.json",
    "Blinding_Lights_lyrics.json",
    "Man_in_the_Mirror_lyrics.json",
    "Stronger_lyrics.json"
]
print("Polarity Scores:")
for file in files:
    polarity = get_lyrics_polarity(file)
    song_name = file.replace("_lyrics.json", "").replace("_", " ")
    if polarity is not None:
        print(f"'{song_name}': {polarity}")
    else:
        print(f"Could not determine polarity for '{song_name}'.")